In [ ]:
import easyocr
import pypdfium2 as pdfium
import cv2
import os
import re
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt
import pdf2image

import torch

reader = easyocr.Reader(['en'])

In [ ]:
torch.cuda.is_available()

In [ ]:
import os

def list_pdf_files(directory):
    pdf_files = []
    for file in os.listdir(directory):
        if file.endswith(".pdf"):
            pdf_files.append(file)
    return pdf_files

# Example usage:
directory_path = os.getcwd()+"\\leases"
pdf_files = list_pdf_files(directory_path)
len(pdf_files)


In [ ]:
os.getcwd()

file = os.getcwd()+"\\leases\\0247_001.pdf"

poppler_path = r"D:\Release-23.08.0-0\poppler-23.08.0\Library\bin"

# images = pdfium.PdfDocument(file)

im = pdf2image.convert_from_path(file, poppler_path=poppler_path)

In [ ]:
im[0]
pic = im[0]
a = np.array(im[15])

b = a[100:1000] #page 1, landlord
c = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
c = cv2.bitwise_not(c)

test = Image.fromarray(c)
test

In [ ]:
results = reader.readtext(c, paragraph = True, batch_size = 4, detail = 0)
full_text = ' '.join([text for (text) in results])
print(full_text)

In [ ]:
#main cell to count the number of 400 forms in the lease directory
z = 0
files_with_400 = [] #initialize list that will get all the file names with 400 forms inside thems
directory_path = os.getcwd()+"\\leases"
for file in os.listdir(directory_path):
    if file.endswith(".pdf"):
        path = os.getcwd()+"\\leases\\"+file
        im = pdf2image.convert_from_path(path, poppler_path=poppler_path)

        for i, image in enumerate(im):
            a = np.array(image)
            b = a[100:1000] #page 1, landlord
            c = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
            c = cv2.bitwise_not(c)
            results = reader.readtext(c, paragraph = True, batch_size = 4, detail = 0)
            full_text = ' '.join([text for (text) in results])
            # print(full_text)
            if "Form 400" in full_text and "Schedule A" not in full_text:
                z += 1
                # print(z, f"400 in file {file} in page {i}")
                files_with_400.append(file)
                break
            else:
                # print(f"no 400 in file {file} in page {i}")
                pass

In [ ]:
import pickle

with open('400_leases_list.pkl', 'wb') as f:
    pickle.dump(files_with_400, f)